In [1]:
#!/usr/bin/python3
# import tensorflow as tf
import numpy as np
from math import exp
from numpy.random import binomial
from random import shuffle
from random import seed
import pandas as pd

# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import CompactLSM

seed(4) # for replicating results

In [2]:
# reading data and defining no.of input neurons, reservoir size, readout neurons
data = pd.read_csv('PreProcessing/PreProcessedData.csv', sep=",", header=None)
data_as_numpy = data.to_numpy()
input = data_as_numpy.transpose()   # Single Sample input
(L,M) = input.shape
# L, M = 77, 100
# input = np.ones([L, M])
# reservoir shape and no.of neurons
nx = 5
ny = 5
nz = 5
N = nx*ny*nz

N_read = 10                                           # No. of Readout neurons
print("Input neurons =",L)
print("Size of Reservoir =",nx,"X",ny,"X",nz,",Total total neurons =",N)
print("Total no.of read out neurons =",N_read)
# print(input [1,])

Input neurons = 78
Size of Reservoir = 5 X 5 X 5 ,Total total neurons = 125
Total no.of read out neurons = 10


In [3]:
global vrest, vth, t_refrac
vrest, vth, t_refrac = 0, 20, 2

tau_m = 32
params_potential = {'C':1, 'g_L':1/tau_m, 'E_L':vrest, 'V_T':vth, 'R_p':t_refrac}

Delay = 1 #constant delay for all synapses in ms

tau_c = 64
C_theta = 5
del_C = 3
n_bits = 3
delta_c = 1
params_conc = {'C_theta':C_theta, 'del_C':del_C, 'tau_c':64, 'nbits':n_bits, 'delta_c':delta_c}

sampling_freq = 12.5 # in khz
h = 1/sampling_freq # in ms

### Finding input neurons to reservoir current and then using the spike train to find the current input to the reservoir

In [4]:
## Connection from input neurons to reservoir
W_in_res = np.zeros((L,N)) # (i,j) entry is the weight of synapse from ith input to jth neuron in reservoir 
W_in = 8
Fin = 4 # no. of neurons a single input neuron is connected to

connection_in_res = np.zeros((L,Fin),dtype=np.int64) # stores the id of reservoir neurons

reservoir_ID = np.array([i for i in range(N)])

for i in range(L):
    shuffle(reservoir_ID)
    for j in range(Fin):
        sign_W_in = (binomial(1,1/2) - 0.5)*2
        W_in_res[i,reservoir_ID[j]] = sign_W_in*W_in
        connection_in_res[i,j] = reservoir_ID[j]


In [5]:
## Current input to the reservoir from the input neurons
In_neurons = input   # spike train of L input neurons, over M timesteps, 1 if spike, 0 if no spike
# print(In_neurons)
In_app = np.zeros((N,M),dtype=np.float64)    # input current to the reservoir.

time = np.array([j*h for j in range(M)],dtype=np.float64)

syn_string = "first-order"

for t in range(M):
    for i in range(L):
        if int(In_neurons[i,t]) == 1:
            for j in range(Fin):
                n_ID = connection_in_res[i,j]
                w_ij = W_in_res[i,n_ID]
                updates = CompactLSM.syn_res(syn_string,1,t,time,i,n_ID,w_ij,Delay,h,M)
#                 print(updates)
                indices = [[n_ID,k] for k in range(M)]
                
                In_app[n_ID,:] += updates
#     print("In_app", In_app[n_ID,:])

# In_app is given as input to the reservoir
# print(connection_in_res)
# for i in range(N): print(str(i), In_app[i,]) 

In [6]:
# Storing the IDs of the neurons
LSM_ID = np.zeros((nx,ny,nz),dtype=np.int64)
l = 0
for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            LSM_ID[i,j,k] = l
            l = l + 1

# Storing the synapse connections, and creating the initial weight matrix
seed(1)
k_prob = [0.45, 0.3, 0.6, 0.15]
r_sq = 2**2

W_arr = [3, 6, -2, -2]
W_init = 3
Weights_temp = np.zeros((N,N))

N_in = int(N*0.8)
neuron_type = [ int(i<N_in) for i in range(N)]
shuffle(neuron_type) # 1 for excitatory, 0 for inhibitory

synapes = [dict() for i in range(N)]    # an array of dictonaries which store the location of neuron, type of neuron, and the IDs of the neurons it is connected to

for l in range(N):
    loc = CompactLSM.ID_to_ind(nx,ny,nz,l)
    n_type = neuron_type[l]
    cons = []
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                if l != int(LSM_ID[i,j,k]):
                    dist_sq = (loc[0]-i)**2 + (loc[1]-j)**2 + (loc[2]-k)**2
                    k_probl = 0
                    if n_type == 1:
                      if neuron_type[int(LSM_ID[i,j,k])] == 1:
                        k_probl = k_prob[0]
                        W_init = W_arr[0]
                      else:
                        k_probl = k_prob[1]
                        W_init = W_arr[1]
                    else:
                      if neuron_type[int(LSM_ID[i,j,k])] == 1:
                        k_probl = k_prob[2]
                        W_init = W_arr[2]
                      else:
                        k_probl = k_prob[3]
                        W_init = W_arr[3]

                    probability = k_probl* exp(-1*dist_sq/r_sq)
#                     print(probability)
                    check = binomial(1,probability)
                    if check == 1:
                        cons.append(int(LSM_ID[i,j,k]))
                        Weights_temp[l,int(LSM_ID[i,j,k])] = W_init    
    synapes[l] = {"Location":loc, "Neuron_type":n_type, "connections":cons}

global Weights
Weights = Weights_temp
# Weights[12,]

### solving the reservoir

In [7]:
# from CompactLSM import reservoir_solver
[Reservoir_potential, Reservoir_Spikes] = CompactLSM.reservoir_solver(N, Delay, synapes, M, h, In_app, params_potential, Weights, syn_string)

In [8]:
All_labels = [str(x) for x in range(10)]
# N_read = 10                                           # No. of Readout neurons
W_init_read = 1                                       # Initial weight, equal for all, update with learning
Weights_temp_readOut = W_init_read*np.ones((N_read, N), dtype='float')


synapes_read = []        # an array of dictonaries which store the label of neuron, 
                                                      # and the IDs of the neurons it is connected to

for l in range(N_read):
    label = All_labels[l]
    synapes_read.append(label)

Weights_readOut = Weights_temp_readOut

In [12]:
Readout_potential, Readout_Spikes, trained_weights = CompactLSM.readOut_response(N_read,N, Delay, synapes, M, h, 
                                                                                 Reservoir_Spikes, params_potential,
                                                                                 params_conc, Weights_readOut, 
                                                                                 syn_string, training=True, 
                                                                                 train_ids=[2])

class_out_label, class_out_idx= CompactLSM.classifier(Readout_Spikes,synapes_read)
Choosen_class_weight = trained_weights[class_out_idx]
print("Predicted Class is",class_out_label)

[3. 3. 8. 3. 3. 3. 3. 3. 3. 3.]
Predicted Class is 2


In [13]:
print(Choosen_class_weight)

[1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
 1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
 1.     1.     1.     1.     1.     1.     1.     0.9999 1.     1.
 1.     1.     1.     1.     1.     1.     1.     1.     1.     0.9999
 1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
 1.     1.     1.     1.     1.     1.     0.9999 1.     1.     1.
 1.0001 1.     1.     1.     1.     1.     1.     1.     1.     1.
 0.9999 1.     1.     1.     1.     1.     1.     1.     1.     0.9999
 1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
 1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
 1.     0.9998 1.     1.     1.     1.     1.     1.     1.     1.
 1.     1.     1.     0.9999 1.     1.     1.     1.     1.     1.
 1.     1.     1.     1.     1.    ]


In [ ]:
## Reservoir spikes
CompactLSM.plot_spikes(Reservoir_Spikes,N,M)


In [ ]:
# input spikes
CompactLSM.plot_spikes(In_neurons,L,M)


In [ ]:
# readout spikes
CompactLSM.plot_spikes(Readout_Spikes,N_read,M)
